# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [8]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
  # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
        
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 加油再等等！中油宣佈下周汽油零售價大降1.3元

▲台灣中油公司今（22）日宣佈下周油價持續調降。（圖／記者林敬旻攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，衝擊全球旅運需求，台灣中油今（22）日宣佈下周汽油每公升零售價調降1.3元，超級柴油調降1.5元，這也是國內油價連續四周下調零售價。
依據中油目前油價估算，調整後的油價分別為92無鉛汽油每公升19元、95無鉛汽油每公升20.5元、98無鉛汽油每公升22.5元、超級柴油每公升16.2元。
國際油價走勢近期波動劇烈，依據中油估算，累算至3月19日之7D3B週均價為28.09美元，較上週33.66下跌5.57美元。新臺幣兌美元匯率30.313較上週30.124貶值0.189元，下周油價持續走跌。
----------------------------------------------------------------------

[1] 謝金河讚他30年來難得好官　沈榮津是「不停進化的2.0版經長」

▲經濟部長沈榮津從基層磨練上來，是財信傳媒董事長謝金河眼中「不停進化的2.0版經濟部沈榮津部長」。（圖／記者湯興漢攝）
記者林淑慧／台北報導
經濟部組建口罩國家隊為防疫付出，財信傳媒董事長謝金河今（22）日在臉書表示，聲浪最高的官員除了衛福部長陳時中、行政院長蘇貞昌，還有已近70歲高齡的經濟部長沈榮津，他行事穩健，待人熱誠，是「不停進化的2.0版經濟部沈榮津部長」，也是他觀察三十多年來，從前部長趙耀東後，難得見到的好官。以下是謝金河臉書全文：
不停進化的2.0版經濟部沈榮津部長
台灣有一段很長的歲月，我們罵官員如駡狗，幾乎到了官不僚生的狀態，很多有才華的人都不想進入公部門當官。但這次台灣抗疫表現得到民眾很高的評價，尤其是陳時中部長，他的沈著，穩健，眼光精準，實事求是，得到大家高度的肯定，也逐漸扭轉大家對政府官員的刻板印象。
這次蘇貞昌領導的內閣是近20年來聲望最高的一次，特別是他要大家不要搶衞生紙，他告訴大家，我們只有一粒「卡臣」，這次大家搶購物資，蘇院長要大家努力買，逆向操作，而且把所有供應商都聯結在一起，這些創意都得到民眾好評。
這當中